<a href="https://colab.research.google.com/github/rrrudolph/trade/blob/master/Strategy_Module.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Format Incoming Data


In [1]:

import pandas as pd
import datetime as dt
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv('drive/My Drive/Colab Notebooks/Data/ETHUSDT_15min.csv')
# df = df[5800:7850]

#### This stuff was to format an original MT4 OHLC file ####

# df['DT'] = df['Date'] + ' '+df['Time']
df['DT'] = pd.to_datetime(df['DT'])
# df = df.set_axis(['Date', 'Time','O','H','L','C','V'], axis=1, inplace=False)
# df.drop(columns=['Date','Time'],inplace=True)
df = df.reindex(columns=['DT', 'O', 'H', 'L', 'C', 'V', 'Bar', 'D_Range',
                         'ADR', 'Pct_Change', 'Frac_H', 'Frac_L', 'Locked_H', 'Locked_L',
                         'Sw_ZZ','Sw_Price','Sw_Rating', 'Sw_Size', 'Sw_Pct', 'Sw_Weight',
                         'TZ_Active', 'TZ_Type', 'TZ_Start',
                         'TZ_End', 'TZ_H', 'TZ_L'])

#dont care:
# df['DT'] -= pd.Timedelta(hours=7) # minus 7 hours to match CST

# set bar type
df['Bar'][df['C'] < df['O']] = 'down'
df['Bar'][df['C'] > df['O']] = 'up'


# if you want to re-iterate over the swings uncomment these:
df['Locked_H'] = 0
df['Locked_L'] = 0

df.reset_index(drop=True,inplace=True)
df


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,DT,O,H,L,C,V,Bar,D_Range,ADR,Pct_Change,Frac_H,Frac_L,Locked_H,Locked_L,Sw_ZZ,Sw_Price,Sw_Rating,Sw_Size,Sw_Pct,Sw_Weight,TZ_Active,TZ_Type,TZ_Start,TZ_End,TZ_H,TZ_L
0,2018-10-25 16:15:00,204.20,204.36,204.00,204.21,824.05478,up,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-10-25 16:30:00,204.21,204.36,204.02,204.24,393.19700,up,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-10-25 16:45:00,204.19,204.29,204.00,204.15,451.13162,down,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-10-25 17:00:00,204.15,204.28,204.07,204.12,591.84310,down,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-10-25 17:15:00,204.12,204.28,204.11,204.14,863.92526,up,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,2018-10-30 20:00:00,196.13,196.36,196.05,196.23,1022.31695,up,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
496,2018-10-30 20:15:00,196.27,196.27,195.91,196.06,1158.66244,down,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
497,2018-10-30 20:30:00,196.09,196.27,196.07,196.19,803.87244,up,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
498,2018-10-30 20:45:00,196.17,196.99,196.17,196.96,944.68991,up,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# ADR

In [2]:

#### get the D_Range and ADR  ####

# To calculate this per day it needs to lag by 24 hrs 
# Otherwise the current day will always look small
# better to just reference a daily df

temp = df[df['DT'] < dt.datetime.today()]  # return all days prior to today (for live data) 
for i in temp['DT'].dt.date.unique():   # iterate through each unique day
    group = df.groupby(df['DT'].dt.date).get_group(i)   # group data by day
    dayrange = max(group['H']) - min(group['L'])   
    df.loc[group.index,'D_Range'] = dayrange

# ADR
#  There are 96 15m candles per day and I want a 5 day average.  
#  This will give me an index error so it needs a try block
##### NOTE: this needs to be modified so it can remain a 5 day
#####       average given any timeframe
for i in temp.index:
    try:
        df.loc[i,'ADR'] = round(df.loc[i-480:i,'D_Range'].mean(),5)
    except:
        num_rows = df.iloc[i].index - df.iloc[0].index
        df.loc[i,'ADR'] = round(df.loc[i - num_rows:i,'D_Range'].mean(),5)

# lastly, fill today's blank ADR values with yesterdays
df['ADR'].fillna(method='ffill', inplace=True)


####  Set Pct_Change  ####
df['Pct_Change'] = df['C'].pct_change()
df

,DT,O,H,L,C,V,Bar,D_Range,ADR,Pct_Change,Frac_H,Frac_L,Locked_H,Locked_L,Sw_ZZ,Sw_Price,Sw_Rating,Sw_Size,Sw_Pct,Sw_Weight,TZ_Active,TZ_Type,TZ_Start,TZ_End,TZ_H,TZ_L
0,2018-10-25 16:15:00,204.20,204.36,204.00,204.21,824.05478,up,1.43,1.43000,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-10-25 16:30:00,204.21,204.36,204.02,204.24,393.19700,up,1.43,1.43000,0.000147,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-10-25 16:45:00,204.19,204.29,204.00,204.15,451.13162,down,1.43,1.43000,-0.000441,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-10-25 17:00:00,204.15,204.28,204.07,204.12,591.84310,down,1.43,1.43000,-0.000147,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-10-25 17:15:00,204.12,204.28,204.11,204.14,863.92526,up,1.43,1.43000,0.000098,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,2018-10-30 20:00:00,196.13,196.36,196.05,196.23,1022.31695,up,2.69,4.80958,0.000612,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
496,2018-10-30 20:15:00,196.27,196.27,195.91,196.06,1158.66244,down,2.69,4.81220,-0.000866,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
497,2018-10-30 20:30:00,196.09,196.27,196.07,196.19,803.87244,up,2.69,4.81482,0.000663,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
498,2018-10-30 20:45:00,196.17,196.99,196.17,196.96,944.68991,up,2.69,4.81744,0.003925,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Fractals

In [0]:

highs = df['H'].values
lows = df['L'].values

# omit rows where the swings are locked
temp = df[(df['Locked_H'] != 1) & (df['Locked_L'] != 1)]


# iterate over the whole df using the unlocked swings
# as starting points in the loop 
for i in temp.index:
    low = lows[i]
    high = highs[i]

    ####  highs  ####
    try:   # iter forwards 
        count_prior = 0   # initialize counters and such
        count_next = 0
        next_ = i + 1
        prior_ = i - 1

        while high > highs[next_] and count_next < 70: # iter limit
            next_ +=1
            count_next +=1
        df.loc[i,'Locked_H'] = 1 # if it hasn't error'd lock the row 
    except:
        pass

    try: # iter backwards
        while high >= highs[prior_] and count_prior < 70: 
            prior_ -=1
            count_prior +=1
    except:
        pass 
    
    frac_h = min([count_prior, count_next])
    df.loc[i,'Frac_H'] = frac_h
    
    try: # the 'min is an empty argument' requires try block
        if  frac_h > 0:  # this will return the swing size if frac > 1
            df.loc[i,'Sw_Size'] = min(lows[i:i+frac_h]) - high
            # idx of lowest low before lockout
            peak = df.loc[i:i+frac_l ,'L']
            df.loc[i, 'Sw_ZZ'] = peak.idxmin()
    except:
        pass

    ####  lows  ####
    try: # iter forwards 
        count_prior = 0
        count_next = 0
        next_ = i + 1
        prior_ = i - 1

        while low < lows[next_] and count_next < 70: 
            next_ +=1
            count_next +=1
        df.loc[i,'Locked_L'] = 1 # if it hasn't error'd set the lock value 
    except:
        pass

    try: # iter backwards
        while low <= lows[prior_] and count_prior < 70: 
            prior_ -=1
            count_prior +=1  
    except:
        pass
    
    frac_l = min([count_prior, count_next])
    df.loc[i,'Frac_L'] = frac_l
    # find swing size (price movement from hi to low or low to hi)
    # only overwrite if the new value is greater 
    try:  
        if frac_l > 0: 
            if frac_l > frac_h:
                df.loc[i,'Sw_Size'] = max(highs[i:i+frac_l]) - low
                # idx of highest high before lockout
                peak = df.loc[i:i+frac_l, 'H']
                df.loc[i, 'Sw_ZZ'] = peak.idxmax()
    except:
        pass

#### Fractals viz

In [4]:
fig = go.Figure(data=[go.Candlestick(x=df['DT'],
                open=df['O'], high=df['H'],
                low=df['L'], close=df['C'])])
# swings = df['Sw_Rating'][df['Sw_Size'] < 0].append(df['Sw_Rating'][df['Sw_Size'] < 0])
hix = df['DT'][df['Sw_Size'] < 0]
lox = df['DT'][df['Sw_Size'] > 0]
hi = df['H'][df['Sw_Size'] < 0]
low = df['L'][df['Sw_Size'] > 0]

ratings = round(df['Frac_H'][df['Sw_Size'] < 0],0)
ratingsl = round(df['Frac_L'][df['Sw_Size'] > 0],0) 

fig.add_trace(go.Scatter(
    x=hix,
    y=hi,
    mode="markers+text",
    name="Swing Ratings",
    text=ratings,
    textposition="top center"
))

fig.add_trace(go.Scatter(
    x=lox,
    y=low,
    mode="markers+text",
    name="Swing Ratings",
    text=ratingsl.values,
    textposition="bottom center"
))

fig.update_layout(
    autosize=False,
    width=1300,
    height=1000,
    margin=dict(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
    # remove this "xaxis" section to show weekend gaps
    xaxis = dict(  
                type="category"),
    paper_bgcolor="LightSteelBlue",  
)

fig.update(layout_xaxis_rangeslider_visible=False)

# Swings

In [5]:
#### Calculate the Swing Ratings #### 


# this is currently deactivated and seems
# not needed now that I've lowered the max
# fractal size



# # get the swing size in relation to ADR
# df['Sw_Pct'] = round(df['Sw_Size']/df['ADR'], 2)

# # create a function of a curved line that will act as a weight
# # to use on the swing size. this will taper off the extreme
# # swing sizes to balance out the swing ratings overall
# df['Sw_Weight'] = 1 -1.02 ** abs(df['Sw_Pct']*100) # function for curved line
# weight = df['Sw_Weight'][abs(df['Sw_Weight'].values) > 0] # list of weights 

# # first fill NaNs with mean of group
# # normalize so its a weight between 0 and 1
# # now that the weights are normalized, multiply the swing by its weight
# # add the fractal rating to its swing and divide by 2

# weight.fillna(value=weight.mean(), inplace=True) 
# df['Sw_Weight'] = (weight - min(weight)) / (max(weight) - min(weight))
# final = df['Sw_Size'] * df['Sw_Weight']
temp = df[abs(df['Sw_Size']) > 0]
#df['Sw_Rating'] = round((abs(final) * 2000 + temp['Frac_H'] + temp['Frac_L']) / 2)
equal_weighted_sw_size = (temp['Frac_H'] + temp['Frac_L']) / abs(df['Sw_Size'])
df['Sw_Rating'] = round((abs(df['Sw_Size']) * equal_weighted_sw_size + temp['Frac_H'] + temp['Frac_L']) / 2)



# df.drop(columns=['Sw_Weight'], inplace=True)

# save the swing prices
for i in df.index:
    a = df.loc[i,'H'][ df.loc[i,'Sw_Size'] < 0]
    b = df.loc[i,'L'][ df.loc[i,'Sw_Size'] > 0]
    if a.size > 0:
        df.loc[i,'Sw_Price'] = a
    elif b.size > 0: 
        df.loc[i,'Sw_Price'] = b

df.head(3)

,DT,O,H,L,C,V,Bar,D_Range,ADR,Pct_Change,Frac_H,Frac_L,Locked_H,Locked_L,Sw_ZZ,Sw_Price,Sw_Rating,Sw_Size,Sw_Pct,Sw_Weight,TZ_Active,TZ_Type,TZ_Start,TZ_End,TZ_H,TZ_L
0,2018-10-25 16:15:00,204.20,204.36,204.00,204.21,824.05478,up,1.43,1.43,NaN,0.0,0.0,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-10-25 16:30:00,204.21,204.36,204.02,204.24,393.19700,up,1.43,1.43,0.000147,3.0,0.0,1,1,1.0,204.36,3.0,-0.36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-10-25 16:45:00,204.19,204.29,204.00,204.15,451.13162,down,1.43,1.43,-0.000441,0.0,2.0,1,1,2.0,204.00,2.0,0.29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Swings viz

In [6]:
fig = go.Figure(data=[go.Candlestick(x=df['DT'],
                open=df['O'], high=df['H'],
                low=df['L'], close=df['C'])])

hix = df['DT'][df['Sw_Size'] < 0]
lox = df['DT'][df['Sw_Size'] > 0]
hi = df['H'][df['Sw_Size'] < 0]
low = df['L'][df['Sw_Size'] > 0]

ratings = round(df['Sw_Rating'][df['Sw_Size'] < 0],0)
ratingsl = round(df['Sw_Rating'][df['Sw_Size'] > 0],0) 

fig.add_trace(go.Scatter(
    x=hix,
    y=hi,
    mode="markers+text",
    name="Swing Ratings",
    text=ratings,
    textposition="top center"
))

fig.add_trace(go.Scatter(
    x=lox,
    y=low,
    mode="markers+text",
    name="Swing Ratings",
    text=ratingsl.values,
    textposition="bottom center"
))

fig.update_layout(
    autosize=False,
    width=1300,
    height=1000,
    margin=dict(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
    # remove this "xaxis" section to show weekend gaps
    xaxis = dict(  
                type="category"),
    paper_bgcolor="LightSteelBlue",  
)

fig.update(layout_xaxis_rangeslider_visible=False)


#### test adding impulse 

In [0]:
# I also want to calculate the impulse
# of the swing (sw_size over time) and add
# that to the rating formula
# df['Impulse'] = 0
# temp = df[df['Sw_Rating'] > 0]
# for i in temp.index:
#     if df.loc[i, 'Sw_Pct'] < 0:
#         fractal = df.loc[i, 'Frac_H']
#     if df.loc[i, 'Sw_Pct'] > 0:
#         fractal = df.loc[i, 'Frac_L']

#     df.loc[i, 'Impulse'] = df.loc[i, 'Sw_Pct']*10000 / fractal

## plot
# fig = go.Figure(data=[go.Candlestick(x=df['DT'],
#                 open=df['O'], high=df['H'],
#                 low=df['L'], close=df['C'])])
# # swings = df['Sw_Rating'][df['Sw_Size'] < 0].append(df['Sw_Rating'][df['Sw_Size'] < 0])
# hix = df['DT'][df['Sw_Size'] < 0]
# lox = df['DT'][df['Sw_Size'] > 0]
# hi = df['H'][df['Sw_Size'] < 0]
# low = df['L'][df['Sw_Size'] > 0]

# ratings = round(df['Impulse'][df['Sw_Size'] < 0],0)
# ratingsl = round(df['Impulse'][df['Sw_Size'] > 0],0) 

# fig.add_trace(go.Scatter(
#     x=hix,
#     y=hi,
#     mode="markers+text",
#     name="Swing Ratings",
#     text=ratings,
#     textposition="top center"
# ))

# fig.add_trace(go.Scatter(
#     x=lox,
#     y=low,
#     mode="markers+text",
#     name="Swing Ratings",
#     text=ratingsl.values,
#     textposition="bottom center"
# ))

# fig.update_layout(
#     autosize=False,
#     width=1300,
#     height=1000,
#     margin=dict(
#         l=50,
#         r=50,
#         b=100,
#         t=100,
#         pad=4
#     ),
#     xaxis = dict(  
#                 type="category"),
#     paper_bgcolor="LightSteelBlue",  
# )

# fig.update(layout_xaxis_rangeslider_visible=False)

# Trades and Trade Zone Viz

In [8]:


active_zones = df[(df['TZ_H'] != np.nan) & # there is a zone
                  (df['TZ_Active'] != False)] 

df['TZ_Active'] = np.nan
df['TZ_Type'] = np.nan
df['TZ_Start'] = np.nan
df['TZ_End'] = np.nan

# save zones for vis
plot_buyx = [] 
plot_buyy = []
plot_buy_flipx = [] 
plot_buy_flipy = []
plot_sellx = [] 
plot_selly = []
plot_sell_flipx = [] 
plot_sell_flipy = []
trade_entries = []
dates = []
names = []

temp =  df[(df['TZ_Active'] != False) &
           (df['Sw_Rating'] > 10)]

for i in temp.index: 

    swing = abs(df.loc[i, 'Sw_Size'])
    zone_height = df.loc[i, 'Sw_Rating'] * df.loc[i,'ADR'] / 75

    # set a limit on zone height
    a = df.loc[2, 'DT'] - df.loc[1, 'DT']  
    b = pd.Timedelta(days = 1)
    cpd = b / a         # candles per day
    max_size = df.loc[i, 'ADR'] / cpd * 42  # wont work for D1 TF 

    if zone_height > max_size:
        zone_height = max_size
    # print("zone height: ", zone_height, "\n max: ", max_size)
    if df.loc[i, 'Sw_Size'] < 0:       # swing high
        df.loc[i,'TZ_Type'] = 'sell'
        df.loc[i,'TZ_H'] = df.loc[i,'Sw_Price'] + zone_height     # upper price 
        df.loc[i,'TZ_L'] = df.loc[i,'Sw_Price']     # lower    
    if df.loc[i, 'Sw_Pct'] > 0:      # swing low
        df.loc[i,'TZ_Type'] = 'buy'
        df.loc[i,'TZ_H'] = df.loc[i,'Sw_Price']     
        df.loc[i,'TZ_L'] = df.loc[i,'Sw_Price'] - zone_height    
    
    ####  set zone end  ####
    end1 = round(df.loc[i, 'Sw_Rating'] * 3.5 + i)   # now get the length of the zone
    if end1 >  len(df) - 1:      # set max value as the length of df
        end1 =  len(df) - 1
    # elif end1 > round(max_size * 10000):
    #     end1 = round(max_size * 10000)
    df.loc[i,'TZ_End'] = end1
   
    ####  set a custom start  #### (makes viz difficult)
    start1 = i  #+ round((end1 - i) * 0.2) 
    # if start1 >  len(df) - 1:
    #     start1 =  len(df) - 1
    df.loc[i,'TZ_Start'] = i #start1

    ####  set buffers  ####  (used for switching zone type)
    df['Buf_Hi'] = df['TZ_H'] + (zone_height * 0.25)
    df['Buf_Lo'] = df['TZ_L'] - (zone_height * 0.25)

    ####  df of candles within current zone  ####
    zone_df = df.loc[start1:end1]
    zone_df = zone_df[(zone_df['H'].between(df.loc[i,'TZ_L'], df.loc[i,'TZ_H'])) |
                      (zone_df['L'].between(df.loc[i,'TZ_L'], df.loc[i,'TZ_H']))]
   
#    for backtesting data where I feed in a whole df, I need to slice 
#    the zones up into parts in order to capture the appropriate trades
#    that would have happened in a bar by bar, live environment
#    to account for breached zones

    ###### SELL ZONES ######
    if df.loc[i,'TZ_Type'] == 'sell':
        if len(zone_df) > 0:
        # if candles are found, look for a candle with lower vol than 
        # original swing, or a series of 3 sequentially lower vol candles
        # but first see if zone gets breached, and if so, shorten the zone_df
        # to end1 where the first breach happens

            # check if zone gets breached to the upside
            new_high = zone_df['H'][zone_df['H'] > df.loc[i, 'Buf_Hi']]
            if len(new_high) > 0: 
                end1 = min(new_high.index)     # set end1 of slice at breach
                zone_df = zone_df.iloc[start1:end1]

            # low volume up bar
            signal_a = zone_df[(zone_df['V'] < df.loc[i,'V']) & (zone_df['Bar'] == 'up')]
            if len(signal_a) > 0:
                # in case of multiple signals, return highest entry price
                sell_price = max(signal_a['O'])
                idx = signal_a['O'].idxmax() # get index of entry candle
                date = zone_df.loc[idx, 'DT']

                # plot first slice
                trade_entries.append(sell_price)
                dates.append(date)
                names.append('sell')
                plot_sellx.append(df.loc[start1,'DT'])
                plot_sellx.append(df.loc[end1,'DT'])
                plot_sellx.append(df.loc[end1,'DT'])
                plot_sellx.append(df.loc[start1,'DT'])
                plot_sellx.append(df.loc[start1,'DT'])
                plot_sellx.append(None)
                plot_selly.append(df.loc[i,'TZ_L'])
                plot_selly.append(df.loc[i,'TZ_L'])
                plot_selly.append(df.loc[i,'TZ_H'])
                plot_selly.append(df.loc[i,'TZ_H'])
                plot_selly.append(df.loc[i,'TZ_L'])
                plot_selly.append(None)

                ### sell flips to buy ###
                # end1 becomes start2 (second slice)
                # (at this point any pending buys should be cancelled)
                if len(new_high) > 0: 
                    # change zone type and look for trades in new zone_df slice
                    df.loc[i,'TZ_Type'] = 'buy' # change zone type
                    end2 = round((end1 - start1) * 0.5 + end1)     # shift end2 forward by 50%
                    start2 = min(new_high.index)    # this is the 'end2' of first slice
                    if end2 > len(df) - 1:
                        end2 = len(df) - 1
                    df.loc[i,'TZ_H'] = df.loc[i,'TZ_H'] - zone_height / 2 # shift zone down by 50%
                    df.loc[i,'TZ_L'] = df.loc[i,'TZ_L'] - zone_height / 2 

                    # reset the zone_df to reflect new zone dimensions
                    flipped_df = df.loc[start2:end2]
                    flipped_df = flipped_df[(flipped_df['H'].between(df.loc[i,'TZ_L'], df.loc[i,'TZ_H'])) |
                                    (flipped_df['L'].between(df.loc[i,'TZ_L'], df.loc[i,'TZ_H']))]
            
            

                    # and look for trades again, but this time buys. if zone
                    # gets breached for a second time it becomes inactive   

                    new_low = flipped_df['L'][flipped_df['L'] < df.loc[i, 'TZ_L']]   
                    if len(flipped_df) > 0:     # first see if any candles exist in new zone
                        if len(new_low) > 0:    # then see if it got breached 
                            idx = min(new_low.index)    # irst candle to breach
                            df.loc[i,'TZ_Active'] = False  
                            # shorten the flipped_df to end2 at that candle
                            # (this won't be relevent for live data, but it's necessary
                            #  to backtest like this. for live data, once the zone
                            #  becomes inactive it will simply be ignored and any
                            #  trades by that point would have already been found)
                            flipped_df = flipped_df.loc[start2:idx]
                        signal_a = flipped_df[(flipped_df['V'] < df.loc[i,'V']) & (flipped_df['Bar'] == 'down')]
                        if len(signal_a) > 0:
                            # in case of multiple signals, get lowest price for buy 
                            buy_price = min(signal_a['O'])
                            idx = signal_a['O'].idxmin()  
                            date = df.loc[idx, 'DT']
                            
                            # plot 
                            trade_entries.append(buy_price)
                            dates.append(date)
                            names.append('buy')            
                            plot_sell_flipx.append(df.loc[start1, 'DT'])
                            plot_sell_flipx.append(df.loc[end2, 'DT'])
                            plot_sell_flipx.append(df.loc[end2, 'DT'])
                            plot_sell_flipx.append(df.loc[start1, 'DT'])
                            plot_sell_flipx.append(df.loc[start1, 'DT'])
                            plot_sell_flipx.append(None)
                            plot_sell_flipy.append(df.loc[i,'TZ_L'])
                            plot_sell_flipy.append(df.loc[i,'TZ_L'])
                            plot_sell_flipy.append(df.loc[i,'TZ_H'])
                            plot_sell_flipy.append(df.loc[i,'TZ_H'])
                            plot_sell_flipy.append(df.loc[i,'TZ_L'])
                            plot_sell_flipy.append(None)
                    
    ###### BUY ZONES ######
    if df.loc[i,'TZ_Type'] == 'buy':
        if len(zone_df) > 0:
        # if candles are found, look for a candle with lower vol than 
        # original swing, or a series of 3 sequentially lower vol candles


            # check if zone gets breached to the downside
            new_low = zone_df['L'][zone_df['L'] < df.loc[i, 'Buf_Lo']]
            if len(new_low) > 0: 
                end1 = min(new_low.index)     # set end1 of slice at breach
                zone_df = zone_df.loc[start1:end1]

            signal_a = zone_df[(zone_df['V'] < df.loc[i,'V']) & (zone_df['Bar'] == 'down')]
            if len(signal_a) > 0:
                # in case of multiple signals, return highest entry price
                buy_price = min(signal_a['O'])
                idx = signal_a['O'].idxmin() # get index of entry candle
                date = zone_df.loc[idx, 'DT']

                # plot 
                trade_entries.append(buy_price)
                dates.append(date)
                names.append('buy')
                plot_buyx.append(df.loc[start1,'DT'])
                plot_buyx.append(df.loc[end1,'DT'])
                plot_buyx.append(df.loc[end1,'DT'])
                plot_buyx.append(df.loc[start1,'DT'])
                plot_buyx.append(df.loc[start1,'DT'])
                plot_buyx.append(None)
                plot_buyy.append(df.loc[i,'TZ_L'])
                plot_buyy.append(df.loc[i,'TZ_L'])
                plot_buyy.append(df.loc[i,'TZ_H'])
                plot_buyy.append(df.loc[i,'TZ_H'])
                plot_buyy.append(df.loc[i,'TZ_L'])
                plot_buyy.append(None)

            ### buy zone flips to sell zone ###
            # second slice (at this point any pending
            # buys should be cancelled)
            if len(new_low) > 0: 
                df.loc[i,'TZ_Type'] = 'sell' # change zone type
                end2 = round((end1 - start1) * 0.5 + end1)     # shift end2 forward by 50%
                start2 = min(new_low.index)    # shift start2 forward to first of breach candles
                if end2 > len(df) - 1:
                    end2 = len(df) - 1
                df.loc[i,'TZ_H'] = df.loc[i,'TZ_H'] + zone_height / 2 # shift zone up by 50%
                df.loc[i,'TZ_L'] = df.loc[i,'TZ_L'] + zone_height / 2 

                # reset the zone_df to reflect new zone dimensions
                flipped_df = df.loc[start2:end2]
                flipped_df = flipped_df[(flipped_df['H'].between(df.loc[i,'TZ_L'], df.loc[i,'TZ_H'])) |
                                (flipped_df['L'].between(df.loc[i,'TZ_L'], df.loc[i,'TZ_H']))]

                # and look for trades again, but this time sells. if zone
                # gets breached for a second time it becomes inactive   

                new_high = flipped_df['H'][flipped_df['H'] > df.loc[i, 'TZ_H']]  # not requiring buffer hit  
                if len(flipped_df) > 0:     # first see if any candles exist in new zone
                    if len(new_high) > 0:    # then see if it got breached 
                        idx = min(new_high.index)    # irst candle to breach
                        df.loc[i,'TZ_Active'] = False  
                        # shorten the flipped_df to end at that candle
                        # (this won't be relevent for live data, but it's necessary
                        #  to backtest like this. for live data, once the zone
                        #  becomes inactive it will simply be ignored and any
                        #  trades by that point would have already been found)
                        flipped_df = flipped_df.loc[start2:idx]
                    signal_a = flipped_df[(flipped_df['V'] < df.loc[i,'V']) & (flipped_df['Bar'] == 'up')]
                    if len(signal_a) > 0:
                        # in case of multiple signals, get highest price for sell 
                        sell_price = max(signal_a['O'])
                        idx = signal_a['O'].idxmax()  
                        date = df.loc[idx, 'DT']
                        
                        # plot 
                        trade_entries.append(sell_price)
                        dates.append(date)
                        names.append('sell')            
                        plot_buy_flipx.append(df.loc[start1, 'DT'])
                        plot_buy_flipx.append(df.loc[end2, 'DT'])
                        plot_buy_flipx.append(df.loc[end2, 'DT'])
                        plot_buy_flipx.append(df.loc[start1, 'DT'])
                        plot_buy_flipx.append(df.loc[start1, 'DT'])
                        plot_buy_flipx.append(None)
                        plot_buy_flipy.append(df.loc[i,'TZ_L'])
                        plot_buy_flipy.append(df.loc[i,'TZ_L'])
                        plot_buy_flipy.append(df.loc[i,'TZ_H'])
                        plot_buy_flipy.append(df.loc[i,'TZ_H'])
                        plot_buy_flipy.append(df.loc[i,'TZ_L'])
                        plot_buy_flipy.append(None)



#### final invalidation of trade zones, by expiry
# if current bar in df is past TZ_End, zone is expired
# df['TZ_Active'][df.index.loc[-1] > df['TZ_End'] = False    # uncomment for live data



fig = go.Figure(data=[go.Candlestick(x=df['DT'],
                open=df['O'], high=df['H'],
                low=df['L'], close=df['C'])])

fig.add_trace(go.Scatter(x=plot_buyx, y=plot_buyy, fill="toself", fillcolor='green', opacity=.4))
fig.add_trace(go.Scatter(x=plot_buy_flipx, y=plot_buy_flipy, fill="toself", fillcolor='rgba(50, 100, 100, 1)', opacity=.4))
fig.add_trace(go.Scatter(x=plot_sellx, y=plot_selly, fill="toself", fillcolor='red', opacity=.4))
fig.add_trace(go.Scatter(x=plot_buy_flipx, y=plot_sell_flipy, fill="toself", fillcolor='rgba(100, 50, 100, 1)', opacity=.4))


fig.add_trace(go.Scatter(
    x=dates,
    y=trade_entries,
    mode="markers+text",
    name="Swing Ratings",
    text=names,
    textposition="top center",
    # fillcolor='blue'
    marker_size=10,
    marker_color='rgba(0, 0, 0, 1)'
    # yaxis= 'y2'
))

fig.update_layout(
    autosize=False,
    width=1300,
    height=1000,
    margin=dict(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
    # remove "xaxis" to show weekend gaps
    xaxis = dict(  
                type="category"),
    paper_bgcolor="LightSteelBlue",  
)

fig.update(layout_xaxis_rangeslider_visible=False)


# Confidence Ratings

#### Trend

for each swing, make a 1:1 projection of length and height for the next swing of the same type.  measure what the next swing actually ended up as and display as a % of original target. if one swing hits the target (creating a new target of equal size) but the following swing come up short that means momentum has faded. 

In [9]:
# .... i think i need to create groups
# of similiar sizes swings....

# get length of previous swing
# only doing swing highs
temp = df[df['Sw_Size'] > 0]
temp = temp.reset_index(inplace=False)
length = []
height = []

for i in temp[2:].index:  # first rows are buffer
    # reference actual idx values of df
    curr = temp.iloc[i, 0]
    prev_1 = temp.iloc[i-1, 0]
    prev_2 = temp.iloc[i-2, 0]
    length = prev_1 - prev_2 
    height = abs(temp.loc[prev_1, 'Sw_Price'] - temp.loc[prev_2, 'Sw_Price'])

    # projected length and time
    # these need to start at the lowest low
    # between swing highs. (Sw_ZZ provides that index)
    zz_idx = df.loc[curr, 'Sw_ZZ']
    pro_length = zz_idx + length 
    pro_height = df.loc[zz_idx, 'Sw_Price'] + height


# I think this needs to be reworked.  i need a df of both
# highs and lows. and I'd also need to group by swing size
# then make AB = CD type projections



KeyError: ignored

instead im going to make it much simpler and see what it reveals. 
create 2 time slices, the first being the last 3 bars and the other bars -10 to -3. get the average in each group of the following data: ratings, bars between swings, and price between highs.

In [0]:
# make the rolling average dynamic.  I want a minimum of 6 swings or 
# a combined sw_rating of 60 (max is 49)
# ...actually just make it 6.

temp = df[df['Sw_Rating'] > 0]
temp = temp.reset_index(inplace=False)

lows = temp[temp['Sw_Pct'] > 0]
highs = temp[temp['Sw_Pct'] < 0]
df_h = highs.tail(3)
df_l = lows.tail(3)



for i in recent.index: 
    ####  highs  ####
    # reference actual idx values of df
    # in case I need to set a value in df
    curr = df_h.iloc[i, 0]
    prev_1 = df_h.iloc[i-1, 0]
    prev_2 = df_h.iloc[i-2, 0]
    prev_3 = df_h.iloc[i-3, 0]

    avg_ratings_h = mean(df_h.loc[curr:prev_3, 'Sw_Rating']) # do i really want this?
   
    # price extensions
    a = df_h[curr,'H'] - df_h[prev_1,'H']
    b = df_h[prev_1,'H'] - df_h[prev_2,'H']
    c = df_h[prev_2,'H'] - df_h[prev_3,'H']
    curr_exten_h = mean([a, b, c]) / (df_h.loc[i,'ADR'] * 100)

    # bars between swings
    a = curr - prev_1
    b = prev_1 - prev_2
    c = prev_2 - prev_3
    curr_freq_h = mean([a, b, c]) / (temp.loc[i,'ADR'] * 100)

    ####  lows  ####
    # indexing
    curr = lows.iloc[i, 0]
    prev_1 = lows.iloc[i-1, 0]
    prev_2 = lows.iloc[i-2, 0]
    prev_3 = lows.iloc[i-3, 0]

    # avg_ratings_l = mean(df_l.loc[curr:prev_3, 'Sw_Rating'])

    # price extensions
    a = df_h[curr,'L'] - df_h[prev_1,'L']
    b = df_h[prev_1,'L'] - df_h[prev_2,'L']
    c = df_h[prev_2,'L'] - df_h[prev_3,'L']
    curr_exten_l = mean([a, b, c]) / (df_l.loc[i,'ADR'] * 100)

    # bars between swings
    a = curr - prev_1
    b = prev_1 - prev_2
    c = prev_2 - prev_3
    curr_freq_l = mean([a, b, c]) / (df_l.loc[i,'ADR'] * 100)

df_h = highs.iloc[-10:-3]
df_l = lows.iloc[-10:-3]

for i in hist.index:

    ####  highs  ####
    # indexing
    prev_3 = highs.iloc[i-3, 0]
    prev_4 = highs.iloc[i-4, 0]
    prev_5 = highs.iloc[i-5, 0]
    prev_6 = highs.iloc[i-6, 0]
    prev_7 = highs.iloc[i-7, 0]
    prev_8 = highs.iloc[i-8, 0]
    prev_9 = highs.iloc[i-9, 0]
    prev_10 = highs.iloc[i-10, 0]

    avg_ratings_h = mean(df_h.loc[prev_3:prev_10, 'Sw_Rating']) # do i really want this?
   
    # price extensions
    # i think this is where I need a lambda function ?
    s = []
    for i in range(3,11):
        a = df_h.loc[prev_{},'H'].format(i) - df_h.loc[prev_{},'H'].format(i+1)
        s.append(a)
    hist_exten_h = mean(s)

    # bars between swings
    s = []
    for i in range(3,11):
        a = prev_{}.format(i) - prev_{}.format(i+1)
        s.append(a)
    hist_freq_h = mean(s)

    ####  lows  ####
    # indexing
    prev_3 = lows.iloc[i-3, 0]
    prev_4 = lows.iloc[i-4, 0]
    prev_5 = lows.iloc[i-5, 0]
    prev_6 = lows.iloc[i-6, 0]
    prev_7 = lows.iloc[i-7, 0]
    prev_8 = lows.iloc[i-8, 0]
    prev_9 = lows.iloc[i-9, 0]
    prev_10 = lows.iloc[i-10, 0]

    avg_ratings_l = mean(df_l.loc[prev_3:prev_10, 'Sw_Rating']) # do i really want this?
   
    # price extensions
    # i think this is where I need a lambda function ?
    s = []
    for i in range(3,11):
        a = df_l.loc[prev_{},'H'].format(i) - df_l.loc[prev_{},'H'].format(i+1)
        s.append(a)
    hist_exten_l = mean(s)

    # bars between swings
    s = []
    for i in range(3,11):
        a = prev_{}.format(i) - prev_{}.format(i+1)
        s.append(a)
    hist_freq_l = mean(s)
    




In [0]:

hist = df.iloc[-10:-3]
df.tail()